# Download all twitter politicians

The following webpage has all the danish politician twitterprofiles  https://filip.journet.sdu.dk/twitter/politikere/

In [1]:
import requests
from bs4 import BeautifulSoup
import re #https://developers.google.com/edu/python/regular-expressions
import pandas as pd

In [2]:
URL = 'https://filip.journet.sdu.dk/twitter/politikere/'
page = requests.get(URL)

In [3]:
soup = BeautifulSoup(page.content, 'html.parser')

In [4]:
soup

<!DOCTYPE html>

<html lang="da">
<head>
<title>twitterpolitikere.dk</title>
<meta charset="utf-8"/>
<meta content="548776584" property="fb:admins">
<meta content="Danske politikere og partier på Twitter." name="description"/>
<meta content="Danske politikere og partier på Twitter." name="og:description"/>
<meta content="Ernst Poulsen og Filip Wallberg" name="author"/>
<meta content="https://filip.journet.sdu.dk/twitter/shared/twitterpolitikere.dk.png" property="og:image">
<meta content="twitterpolitikere.dk" property="og:title">
<meta content="twitterpolitikere.dk" property="og:site_name">
<meta content="summary" name="twitter:card"/>
<meta content="twitterpolitikere.dk" name="twitter:domain"/>
<link href="https://filip.journet.sdu.dk/twitterpolitikere.dk" rel="canonical"/>
<meta content="https://filip.journet.sdu.dk/twitterpolitikere.dk" property="og:url"/>
<meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
<link href="https://filip.journet.sdu.dk

## Selecting the twitternames

See that the twitter names is in a h3 style:

In [5]:
twitter_names = soup.find_all('h3')

In [6]:
#twitter_names

In [7]:
twitter_name_list=[]

In [8]:
for names in twitter_names:
   # print(names, end='\n'*2)
    match=re.search("<h3>#",str(names))
    if match:
        result_name=re.findall(".com/\w*",str(names))[0].replace(".com/","")
        twitter_name_list.append(result_name.rstrip())
        print(result_name)

vestager
larsloekke
oestergaard
PSkipperEL
IdaAuken
Kristian_Jensen
uffeelbaek
SorenPape
PiaOlsen
Heunicke
sofiecn
regeringDK
Statsmin
brianholm1962
ammitzbollbille
DanJoergensen
JakobEllemann
DanishMFA
mattiastesfaye
EllenTraneNorby
RosenkrantzT
MrMesserschmidt
FrankJensenKBH
aahlers
sophieloehde
martinlidegaard
karmel80
zeniastampe
Astridkrag
BennyEngelbrech
PHummelgaard
RasmusJarlov
JanEJoergensen
StemLAURITZEN
MogensJensenS
metteabildgaard
mfMorten
PerClausen3
Kristianthdahl
RasmusPrehn
_MaiMercado
jacobmark_sf
Jesper_Pet
nvillumsen
Trinebramsen
skaarup_df
SosseSass
MargreteAuken
MetteGjerskov
AlexVanopslagh
rohde_jens
LotteRod
mortenhelveg
UFM_MIN
espersendf
PernilleVermund
olebirkolesen
Pia_Kjaersgaard
RosaLundEl
LisbethBNielsen
rasmusnordqvist
JeppeKofod
SophieHAndersen
AneHalsboe
SchaldemoseMEP
Loekkegaard_MEP
Finansmin
engelschmidt
simonkollerup
KHegaard
khoenge
Jens_Joel
Erhvervs_min
Ulla_Tornaes
Torstenschack
rasmushorn
michaelaastrup
DKsundhed
KofodPeter
naserkhaderdk
MFVMi

## The real names

The real names is in a < small > class.

In [9]:
real_name_list=[]

In [10]:
real_names = soup.find_all('small')

In [11]:
for r_names in real_names:
    result=re.search(r'<small>.+',str(r_names)).group(0).replace("<small>","")
    real_name_list.append(result.rstrip())
    print(result)

Margrethe Vestager 
Lars Løkke Rasmussen 
Morten Østergaard 
Pernille Skipper 
Ida Auken 
Kristian Jensen 
Uffe Elbaek 
Søren Pape Poulsen 
Pia Olsen Dyhr 
Magnus Heunicke 
Sofie CarstenNielsen 
Regeringen 
Statsministeriet 
Brian Holm 
Simon Emil Ammitzbøll-Bille 
Dan Jørgensen 
Jakob Ellemann-Jensen 
Denmark MFA 🇩🇰 
Mattias Tesfaye 
Ellen Trane Nørby 
Rosenkrantz-Theil 
Morten Messerschmidt 
Frank Jensen 
Tommy Ahlers 
Sophie Løhde 
Martin Lidegaard 
Karen Melchior😷🧼🇪🇺 
Zenia Stampe 
Astrid Krag 
Benny Engelbrecht 
Peter Hummelgaard 
Rasmus Jarlov 
Jan E. Jørgensen 
Karsten Lauritzen 
Mogens Jensen 
Mette Abildgaard 
Morten Bødskov 
Per Clausen 
Kristian Thulesen Dahl 
Rasmus Prehn 
Mai Mercado 
Jacob Mark 
Jesper Petersen 
Nikolaj Villumsen 
Trine Bramsen 
Peter Skaarup 
Henrik Sass 
Margrete Auken 
Mette Gjerskov 
Alex Vanopslagh 
Jens Rohde 
Lotte Rod 
Morten Helveg 
Uddannelses- og Forskningsministeriet 
Søren Espersen 
Pernille Vermund 
Ole Birk Olesen 
Pia Kjærsgaard 
Rosa Lund

In [12]:
len(real_name_list)

650

## Making dataframe

In [13]:
d={'Names':real_name_list,"Twittername":twitter_name_list}
data=pd.DataFrame(data=d)

In [17]:
data.to_csv("danish_politkere_twitter.csv")

In [18]:
data.head()

,Names,Twittername
0,Margrethe Vestager,vestager
1,Lars Løkke Rasmussen,larsloekke
2,Morten Østergaard,oestergaard
3,Pernille Skipper,PSkipperEL
4,Ida Auken,IdaAuken


## Finding policial parties from https://www.danskepolitikere.dk/

In [24]:
soup = BeautifulSoup(requests.get('https://www.danskepolitikere.dk/oversigt/danske-politikere').content, 'html.parser')


In [25]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="da">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=480" name="viewport"/>
  <title>
   Danske Politikere indeks
  </title>
  <meta content="jilW832BtrKNfbLwBTR6nXfXyHMHUgfaqYJ1Zo7ALYc" name="google-site-verification">
   <link href="/assets/frontend-5f9e3e1cdd94cf755ff9f893c0c1fdc7.css" media="all" rel="stylesheet"/>
   <link href="//maxcdn.bootstrapcdn.com/font-awesome/4.2.0/css/font-awesome.min.css" rel="stylesheet"/>
   <script async="async" src="/assets/frontend-4dbd56155397f55964891167e9b69119.js">
   </script>
   <meta content="authenticity_token" name="csrf-param">
    <meta content="xxxRsxs3hy1utVCCI+EneniY8vUBLMfm1SES35kACUE=" name="csrf-token"/>
   </meta>
  </meta>
 </head>
 <body class="alle-politikere" id="alle-politikere">
  <div id="service-wrapper">
   <div id="service">
    <div id="alerter">
     <p>
      <a href="mailto:kontakt@danskepolitikere.dk">
       Kommentarer eller rettelser? Skriv til os!
       <i class="fa fa-envelo

In [27]:
sections = list(soup.find_all('section'))

In [62]:
# add "UNKNOWN" as default party
data['Party'] = 'UNKNOWN'
data.head()


'UNKNOWN'

 z


In [79]:
parties = []
# cross reference for our politicians and check if he's in the party
for row, coloumns in data.iterrows(): 
    name = coloumns[0] 
    for section in sections:
        party = section.h3.a.text # gets the party 
        if re.search(name.lower(), str(section).lower()):
            data.at[row, 'Party'] = party
            print("Found " + name + " in " + party)
            
        


Found Margrethe Vestager in Radikale Venstre
Found Lars Løkke Rasmussen in Venstre
Found Morten Østergaard in Radikale Venstre
Found Pernille Skipper in Enhedslisten
Found Ida Auken in Radikale Venstre
Found Kristian Jensen in Venstre
Found Søren Pape Poulsen in Det Konservative Folkeparti
Found Pia Olsen Dyhr in Socialistisk Folkeparti
Found Magnus Heunicke in Socialdemokratiet
Found Statsministeriet in Radikale Venstre
Found Statsministeriet in Socialdemokratiet
Found Statsministeriet in Venstre
Found Dan Jørgensen in Socialdemokratiet
Found Jakob Ellemann-Jensen in Venstre
Found Mattias Tesfaye in Socialdemokratiet
Found Ellen Trane Nørby in Venstre
Found Rosenkrantz-Theil in Socialdemokratiet
Found Morten Messerschmidt in Dansk Folkeparti
Found Frank Jensen in Socialdemokratiet
Found Tommy Ahlers in Venstre
Found Sophie Løhde in Venstre
Found Martin Lidegaard in Radikale Venstre
Found Zenia Stampe in Radikale Venstre
Found Astrid Krag in Socialdemokratiet
Found Benny Engelbrecht in

In [80]:
data.groupby('Party').count()

,Names,Twittername
Party,,
Alternativet,1,1
Dansk Folkeparti,13,13
Det Konservative Folkeparti,14,14
Enhedslisten,12,12
Inuit Ataqatigiit,1,1
Liberal Alliance,5,5
Radikale Venstre,15,15
Socialdemokratiet,42,42
Socialistisk Folkeparti,19,19


In [81]:
data.loc[data['Party'] == "UNKNOWN"]


,Names,Twittername,Party
6,Uffe Elbaek,uffeelbaek,UNKNOWN
10,Sofie CarstenNielsen,sofiecn,UNKNOWN
11,Regeringen,regeringDK,UNKNOWN
13,Brian Holm,brianholm1962,UNKNOWN
14,Simon Emil Ammitzbøll-Bille,ammitzbollbille,UNKNOWN
...,...,...,...
645,Molle lykke nielsen,LykkeMolle,UNKNOWN
646,Vibeke Gamst,vibeke_gamst,UNKNOWN
647,Pia Moldt,piamoldt,UNKNOWN
648,Lars Folmann,larsfolm,UNKNOWN
